In [1]:
import numpy as np
from scipy.linalg import sqrtm, logm
import mne

def print_if_complex(matrix, name):
    if np.iscomplexobj(matrix):
        print(f"{name} contains complex values:\n", matrix)

class GenerateurDonnees:
    def __init__(self, channels=3, sfreq=256.0, duration=10, change=5.0, A1=2.0e-6, A2=0.0e-6, sd=2.0e-6):
        self.channels = channels
        self.sfreq = sfreq
        self.duration = duration
        self.change = change
        self.frequencies = 10.0 + 2.0 * np.random.rand(channels)
        self.phases = 2.0 * np.pi * np.random.rand(channels)
        self.A1 = A1
        self.A2 = A2
        self.sd = sd
        self.info = mne.create_info([f'EEG{n:03}' for n in range(1, channels + 1)],
                                    ch_types=['eeg'] * channels, sfreq=sfreq)

    def generer_donnees(self):
        samples = int(self.sfreq * self.duration)
        t = np.linspace(0, self.duration, samples, endpoint=False)
        A = self.A1 + (t > self.change).astype(float) * (self.A2 - self.A1)
        data = A * np.cos(np.outer(self.frequencies, t) + np.outer(self.phases, np.ones(samples)))
        data += np.random.normal(0, self.sd, size=(self.channels, samples))
        # Vérification des valeurs infinies ou NaN
        if not np.isfinite(data).all():
            raise ValueError("Les données générées contiennent des valeurs infinies ou NaN")
        raw = mne.io.RawArray(data, self.info)
        return raw

    def exporter_donnees(self, filename='dummy.edf'):
        raw = self.generer_donnees()
        mne.export.export_raw(filename, raw, fmt='edf', overwrite=True)

class CalculCovariance:
    def __init__(self, p_points, stride):
        self.p_points = p_points
        self.stride = stride
    
    def calculer_matrices_covariance(self, donnees):
        k, n = donnees.shape
        matrices_covariance = []
        
        for start_index in range(0, n - self.p_points + 1, self.stride):
            end_index = start_index + self.p_points
            segment = donnees[:, start_index:end_index]
            covariance_matrix = np.cov(segment) + np.eye(k) * 1e-6  # Régularisation ajoutée
            matrices_covariance.append(covariance_matrix)
        
        return matrices_covariance

class Transformation:
    def __init__(self):
        pass

    def project_to_tangent(self, cov_matrix_current, cov_matrix_next):
        sqrt_B = sqrtm(cov_matrix_next)
        print_if_complex(sqrt_B, "sqrt_B")
        sqrt_inv_B = np.linalg.inv(sqrt_B)
        print_if_complex(sqrt_inv_B, "sqrt_inv_B")
        transformed_A = np.dot(np.dot(sqrt_inv_B, cov_matrix_current), sqrt_inv_B)
        print_if_complex(transformed_A, "transformed_A")
        log_transformed_A = logm(transformed_A)
        print_if_complex(log_transformed_A, "log_transformed_A")
        return log_transformed_A

    def transport_to_tangent(self, delta_matrix, tangent_plane_start, tangent_plane_end):
        inv_B = np.linalg.inv(tangent_plane_end)
        print_if_complex(inv_B, "inv_B")
        A_invB = np.dot(tangent_plane_start, inv_B)
        print_if_complex(A_invB, "A_invB")
        E = sqrtm(A_invB)
        print_if_complex(E, "E")
        transported_Delta = np.dot(np.dot(E, delta_matrix), E.T)
        print_if_complex(transported_Delta, "transported_Delta")
        return transported_Delta

class CovariancePipeline:
    def __init__(self, data, oublie):
        self.data = data
        self.cov_matrices = []
        self.trajectory = []
        self.transformation = Transformation()
        self.oublie = oublie

    def compute_covariances(self):
        for i in range(len(self.data)):
            cov_matrix = self.data[i]
            self.cov_matrices.append(cov_matrix)


        return self.trajectory
    def compute_trajectory(self):
        for i in range(len(self.cov_matrices) - 1):
            cov_matrix_current = self.cov_matrices[i]
            cov_matrix_next = self.cov_matrices[i + 1]
            
            # Projection de la matrice actuelle vers le plan tangent de la matrice suivante
            projected_matrix = self.transformation.project_to_tangent(cov_matrix_current, cov_matrix_next)
            print_if_complex(projected_matrix, f"projected_matrix {i}")
            self.trajectory.append(projected_matrix)
            # Si c'est la première projection, elle est ajoutée directement
            if not self.trajectory:
                self.trajectory.append(projected_matrix)
            else:
                # Nouvelle trajectoire à ajouter
                new_trajectory = projected_matrix
                
                # Transport de toutes les projections actuelles vers le plan tangent de la matrice suivante
                for j in range(len(self.trajectory)):
                    self.trajectory[j] = self.transformation.transport_to_tangent(self.trajectory[j], cov_matrix_current, cov_matrix_next) + new_trajectory
                    print_if_complex(self.trajectory[j], f"transported trajectory[{j},  {i}] after transport")
                    # Additionner la matrice transportée à la matrice projetée
                    new_trajectory += self.trajectory[j]
                
                # Limiter la taille de la trajectoire à p
                if len(self.trajectory) > self.oublie:
                    self.trajectory.pop(0)

        return self.trajectory

# Exemple d'utilisation
# Générer des données initiales
generateur = GenerateurDonnees()
raw_data_initiale = generateur.generer_donnees()
donnees_initiales = raw_data_initiale.get_data()

# Calculer les matrices de covariance initiales
calcul_cov = CalculCovariance(p_points=256, stride=128)
matrices_covariance_initiales = calcul_cov.calculer_matrices_covariance(donnees_initiales)

pipeline = CovariancePipeline(matrices_covariance_initiales, oublie=5)
pipeline.compute_covariances()
trajectory = pipeline.compute_trajectory()

print(trajectory)


Creating RawArray with float64 data, n_channels=3, n_times=2560
    Range : 0 ... 2559 =      0.000 ...     9.996 secs
Ready.
[array([[ 3480.94024962,  4782.94852891,  7755.04758871],
       [ 4782.94852928,  1082.84970008, -1036.96604702],
       [ 7755.04758902, -1036.96604708,  -332.91375093]]), array([[14011.06487057, 19251.75328446, 31214.69583565],
       [19251.75328595,  4358.55618997, -4173.87589156],
       [31214.69583689, -4173.8758918 , -1340.00479761]]), array([[ 49433.77845944,  67923.95172444, 110131.56880969],
       [ 67923.9517297 ,  15377.83505035, -14726.25700552],
       [110131.56881409, -14726.25700636,  -4727.79901266]]), array([[157029.0097414 , 215764.01141481, 349838.76604149],
       [215764.01143153,  48848.50239494, -46778.74217069],
       [349838.76605546, -46778.74217338, -15018.11601182]]), array([[ 449294.67446744,  617348.84794405, 1000966.4309421 ],
       [ 617348.84799188,  139766.41606348, -133844.49200188],
       [1000966.43098208, -133844.492